In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOllama

import streamlit as st
import ollama
from typing import Dict, Generator
from glob import glob



### LLM 🤓

def ollama_generator(messages: Dict) -> Generator:
    stream = ollama.chat(
            model="llava:7b",
            messages=messages,
            stream=True
        )
    for chunk in stream:
        yield chunk['message']['content']



### Data Integration
folder_path= '/Users/rayaneghilene/Documents/Ollama/RAG/Famso-Data'

# @st.cache_resource
def load_pdf():
    # pdf_name ='Issues with Entailment-based Zero-shot Text Classification.pdf'
    #loaders = [PyPDFLoader(pdf_name)]
    pdf_files = glob(f"{folder_path}/*.pdf")
    loaders = [PyPDFLoader(file_path) for file_path in pdf_files]

    index= VectorstoreIndexCreator(
        embedding = HuggingFaceEmbeddings(model_name= 'all-MiniLM-L12-V2'),
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
    ).from_loaders(loaders)
    return index

index = load_pdf()


### Chain

chain= RetrievalQA.from_chain_type(
    llm= ChatOllama(model="llava:7b"),
    chain_type ='stuff',
    retriever= index.vectorstore.as_retriever(),
    input_key='question'
)

/Users/rayaneghilene/Documents/Ollama/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/rayaneghilene/Documents/Ollama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
question = "What is zero-shot text classification?"
response = chain({"question": question})
print(response)

/Users/rayaneghilene/Documents/Ollama/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'question': 'What is zero-shot text classification?', 'result': " La classificación de Lancefield appartient à la famille des streptococci (Streptococcaceae) et est un groupe d'espèces gram négatives, diplocoques en forme de coccoïdes. En revanche, le genre Coccus comprend des bactéries gram positives, principalement en forme de coccus.\n\nZero-shot text classification est une technique de machine learning qui permet de classer un texte sur une catégorie à partir d'un vocabulaire limité, c'est-à-dire qu'il n'y a pas besoin de l'entraînement sur des exemples spécifiques de la catégorie. Cette technique peut être utilisée pour classer un texte sur une classe que le modèle n'a jamais rencontré auparavant, mais qui est similaire à celles qu'il connaît déjà. "}
